In [441]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
import os
import warnings
# 禁用警告
warnings.filterwarnings("ignore")
device="cuda"
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import gc
import time
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import GroupKFold
NUM_WORKERS = 4
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
from datetime import datetime


In [442]:
# The tech stocks we'll use for this analysis
tech_list = ['AAPL', 'GOOG', 'MSFT', 'AMZN']
end = datetime.now()
start = datetime(end.year -3, end.month-0, end.day)

for stock in tech_list:
    globals()[stock] = yf.download(stock, start, end)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [443]:
company_data = [AAPL, GOOG, MSFT, AMZN]
company_name = ["APPLE", "GOOGLE", "MICROSOFT", "AMAZON"]
for company_data, com_name in zip(company_data, company_name):
    company_data["company_name"] = com_name
    
# df = pd.concat(company_list, axis=0)
# df.tail(10)


In [444]:

AAPL_Close_df = AAPL.filter(['Close'])
print(type(AAPL_Close_df))
print(AAPL_Close_df.shape)
AAPL_Close =AAPL_Close_df.values
print(type(AAPL_Close))
print(AAPL_Close.shape)


<class 'pandas.core.frame.DataFrame'>
(755, 1)
<class 'numpy.ndarray'>
(755, 1)


In [445]:
# 归一化
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
AAPL_Close = scaler.fit_transform(AAPL_Close)

AAPL_Close
print(AAPL_Close.shape)


(755, 1)


In [446]:
# Split the data into x_train and y_train data sets
x_train = []
y_train = []
num_sample = 10
training_data_len = int(np.ceil(len(AAPL_Close)*0.8))
for i in range(num_sample,training_data_len):
    print(i)
    x_train.append(AAPL_Close[i-num_sample:i])
    y_train.append(AAPL_Close[i])
    print(AAPL_Close[i-num_sample:i].shape,AAPL_Close[i])

x_train, y_train = np.array(x_train), np.array(y_train)
print(x_train.shape,y_train.shape)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))
print(x_train.shape,y_train.shape,type(x_train),type(y_train))

# # dataset = np.column_stack((data, labels))
# dataset = np.column_stack((x_train, y_train))


10
(10, 1) [0.10405764]
11
(10, 1) [0.08824552]
12
(10, 1) [0.09533772]
13
(10, 1) [0.08022316]
14
(10, 1) [0.03964651]
15
(10, 1) [0.05499356]
16
(10, 1) [0.0649924]
17
(10, 1) [0.07150325]
18
(10, 1) [0.10010465]
19
(10, 1) [0.14277409]
20
(10, 1) [0.14695966]
21
(10, 1) [0.14533195]
22
(10, 1) [0.1373096]
23
(10, 1) [0.15474943]
24
(10, 1) [0.16207413]
25
(10, 1) [0.1318451]
26
(10, 1) [0.14881986]
27
(10, 1) [0.13916989]
28
(10, 1) [0.1318451]
29
(10, 1) [0.20276707]
30
(10, 1) [0.20195322]
31
(10, 1) [0.21230084]
32
(10, 1) [0.18858275]
33
(10, 1) [0.20683635]
34
(10, 1) [0.24927338]
35
(10, 1) [0.23857697]
36
(10, 1) [0.25031973]
37
(10, 1) [0.30519709]
38
(10, 1) [0.28403667]
39
(10, 1) [0.2706662]
40
(10, 1) [0.25869087]
41
(10, 1) [0.22055578]
42
(10, 1) [0.23915816]
43
(10, 1) [0.1878851]
44
(10, 1) [0.23811181]
45
(10, 1) [0.25124988]
46
(10, 1) [0.21555627]
47
(10, 1) [0.21346357]
48
(10, 1) [0.23776303]
49
(10, 1) [0.2147425]
50
(10, 1) [0.19416345]
51
(10, 1) [0.2021858]


In [447]:

test_data = AAPL_Close[training_data_len-num_sample:]
# print(test_data)
x_test = []
y_test = []
for i in range(num_sample, len(test_data)):
    # print(i)
    # print(test_data[i-10:i])
    x_test.append(test_data[i-num_sample:i])
    y_test.append(test_data[i])
    
# Convert the data to a numpy array
x_test,y_test = np.array(x_test),np.array(y_test)
# print(x_test)
# print(x_test.shape,y_test.shape)
# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))
y_test = np.reshape(y_test, (y_test.shape[0], 1 ))
print(x_test.shape,y_test.shape)
# print(x_test)

(151, 10, 1) (151, 1)


In [448]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data  # 输入数据数组，形状为(670, 10, 1)
        self.labels = labels  # 对应标签数组，形状为(670, 1)
        print(data.shape)
        print(type(data))
        print(labels.shape)
    def __len__(self):
        return len(self.data)

    # self.prepare_data(self):
    #     pass


    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx], dtype=torch.float)
        y = torch.tensor(self.labels[idx], dtype=torch.float)
        return x, y

In [449]:
# dataset = MyDataset(data, labels)
train_dataset = MyDataset(x_train, y_train)
val_dataset = MyDataset(x_test, y_test)


# print(len(train_dataset))
# print(train_dataset[0][0].shape)# 数据   dataset[第几个样本][0 数据  1标签]
# print(train_dataset[0][1].shape)# 对应标签

(594, 10, 1)
<class 'numpy.ndarray'>
(594, 1)
(151, 10, 1)
<class 'numpy.ndarray'>
(151, 1)


In [702]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=False,
    drop_last=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    drop_last=False
)

In [774]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(1, 1)
        )
        self.lstm = nn.LSTM(
            input_size=1, 
            hidden_size=128, 
            batch_first=True, 
            bidirectional=True)

        self.lstm = nn.LSTM(
            input_size=256, 
            hidden_size=64, 
            batch_first=True, 
            bidirectional=True)
        
        # 添加一个全连接层
        self.fc1 = nn.Sequential(
            nn.Linear(64 * 2, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

        # 添加一个全连接层
        self.fc2 = nn.Sequential(
            nn.ReLU(),
            nn.Linear(10, 1)
        )
    def forward(self, x):
        print(x.shape)
        features = self.mlp(x)
        print(features.shape)
        lstm_out, _ = self.lstm(features)
        last_lstm_output = lstm_out[:, -1, :]  # 仅获取最后一个时间步的输出
        # print('lstm_out',lstm_out.shape)
        # print('last_lstm_output',last_lstm_output.shape)
        fc1_out = self.fc1(last_lstm_output)
        # print('fc1_out',fc1_out.shape)
        # fc1_out = fc1_out.squeeze(-1)
        # print('fc1_out',fc1_out.shape)
        # out = self.fc2(fc1_out)
        # print('out',out.shape)
        return fc1_out



In [775]:
model = RNNModel()

In [776]:
lr = 0.0000003
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),lr=lr)


In [777]:
epochs = 20
for epoch in range(epochs):
    preds = []
    model.train()
    model.zero_grad()
    avg_loss = 0
    start_time = time.time()
    for id,(data,label) in enumerate(train_loader):
        print(id,(data.shape,label.shape))
        pred = model(data)
        # print(pred.shape)
        # pred = pred.squeeze(-1)
        # print('pred',pred.shape)
        # print('label',label.shape)
        loss = criterion(pred,label)
        # print(pred,label)
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(label)
        
        break


    # model.eval()
    # avg_val_loss = 0
    

    # with torch.no_grad():
    #     for id,(data,label) in enumerate(val_loader):
    #         # print(id,(data.shape,label.shape))
    #         pred = model(data)
    #         # print(pred.shape)
    #         # pred = pred.squeeze(-1)
    #         # print(pred.shape)
    #         # print(label.shape)
    #         loss = criterion(pred,label)
    #         # print(loss)
    #         # optimizer.zero_grad()
    #         # loss.backward()
    #         # optimizer.step()
    #         avg_val_loss += loss.item() / len(label)
    #         # break

    #         preds.extend(pred.detach().cpu().numpy())
    #         # print(np.array(preds).shape)

    # preds = np.concatenate(preds, 0)[:,None]
    
    elapsed_time = time.time() - start_time
    
    elapsed_time = elapsed_time

    
    


    print(
        f"train Epoch {epoch + 1:02d}/{epochs:02d}  lr={lr:.1e}\t t={elapsed_time:.0f}s  "
        f"loss={avg_loss:.8f}  val_loss={avg_val_loss:.8f}")


    


0 (torch.Size([16, 10, 1]), torch.Size([16, 1]))
torch.Size([16, 10, 1])
torch.Size([16, 10, 1])


RuntimeError: input.size(-1) must be equal to input_size. Expected 256, got 1

In [ ]:
predictions = scaler.inverse_transform(preds)
# Plot the data
train = AAPL_Close_df[:training_data_len+1]
valid = AAPL_Close_df[training_data_len:]

valid['Predictions'] = predictions

# Visualize the data
plt.figure(figsize=(16,6))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(AAPL_Close_df['Close'],'o')
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['yuan ','Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.